The following program reads in a file called raw_data.csv

date: datetime
visitID: any format, will be converted to string on import
deviceType: text
channel: text
experimentBucket: A or B
maximumSavingsSeen: decimal
switched: any format, will be changed to object

You also need to define the bin boundaries in Stage 2.2 and hypothesis for Experiment C in Stage 7.

In [ ]:
# bin boundaries for Stage 2.2
a = 100.0000000
b = 200.0000000
c = 300.0000000
d = 400.0000000
e = 500.0000000

# 1. Import raw data, explore and normalise

## 1.1. Import libraries and raw data, normalise raw data

In [ ]:
import os
import pandas as pd
import numpy as np
import scipy

# pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# for boxplot
from pylab import *

# use the inline backend to generate the plots within the browser
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors


# import seaborn as sns
# sns.set_style('whitegrid')cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for identifying histogram bins - import after first matplotlib histogram
# import seaborn as sns
# sns.set_style('whitegrid')

print('Libraries imported.')

In [ ]:
# function to export a datafrane to a csv file within a new or existing directory

def export_df_csvdir (df, exp_dir, exp_csv):
    export_dir1 = './exports1'
    if not os.path.exists(exp_dir):
        os.mkdir(exp_dir)
    exp_dir_csv = os.path.join(exp_dir, exp_csv)
    df.to_csv (exp_dir_csv)
       
# function to export a datafrane to an Excel file and sheet within a new or existing directory

def export_df_xlsdir (df, exp_dir, exp_xls, xls_sheet):
    export_dir1 = './exports1'
    if not os.path.exists(exp_dir):
        os.mkdir(exp_dir)
    exp_dir_xls = os.path.join(exp_dir, exp_xls)
    df.to_excel (exp_dir_xls, sheet_name= xls_sheet) 

In [ ]:
# read in raw data, cast 'visitID' as object to ensure not summed, cast 'switched' as integer as required as flter
all_visits_df = pd.read_csv('raw_data.csv', dtype={'visitID': object, 'switched': int})

# drop index column as it is duplicate of 'visitID'
all_visits_df.drop(columns = ['Unnamed: 0'], inplace = True)

# add count
all_visits_df ['totalVisits'] = 1

# remove visits outside experiement buckets A & B
all_ab_visits_df = all_visits_df [all_visits_df ['experimentBucket']!='None']

In [ ]:
all_ab_visits_df.info()

In [ ]:
all_ab_visits_df.head()

## 2. Examine distribution of 'maximumSavingsSeen'

## 2.1 Outliers

In [ ]:
all_ab_visits_df['maximumSavingsSeen'].describe()

In [ ]:
# show boxplot with outliers
boxplot(all_ab_visits_df['maximumSavingsSeen']) #, showfliers=False)
show()
#sns.set_style('whitegrid')
hist = all_ab_visits_df['maximumSavingsSeen'].hist(bins=1000)

In [ ]:
all_ab_visits_df['z-score'] = scipy.stats.zscore(all_ab_visits_df['maximumSavingsSeen'])

In [ ]:
all_ab_visits_df['z-score'] = np.abs(all_ab_visits_df['z-score'])

In [ ]:
all_ab_visits_df.head()

In [ ]:
all_ab_visits_df['outlier'] = all_ab_visits_df['z-score'].apply(lambda x: False if x < 3 else True) 

In [ ]:
all_ab_visits_df.head()

In [ ]:
all_ab_visits_no_outliers_df = all_ab_visits_df[all_ab_visits_df['outlier'] == False]

In [ ]:
all_ab_visits_no_outliers_df.info()

In [ ]:
# NOTE: have chosen not to filter out negative maximumSavingsSeen as there may be rises in wholesale prices for example
# all_ab_visits_no_outliers_df = all_ab_visits_df[all_ab_visits_df['maximumSavingsSeen'] > 0]

## 2.2 Binning

In [ ]:
import seaborn as sns

sns.set_style('whitegrid')

all_ab_visits_no_outliers_df['maximumSavingsSeen'].plot(kind='hist')

In [ ]:
all_ab_visits_no_outliers_df['maximumSavingsSeen'].describe()

In [ ]:
hist = all_ab_visits_no_outliers_df['maximumSavingsSeen'].hist(bins=5)

In [ ]:
def bin (amount):
    if amount < a:
        return "bin1"
    elif amount < b:
        return "bin2"
    elif amount < c:
        return "bin3"
    elif amount < d:
        return "bin4"
    elif amount < e:
        return "bin5"
    
all_ab_visits_no_outliers_df['bin'] = all_ab_visits_no_outliers_df.apply \
                    (lambda x: bin(x['maximumSavingsSeen']),axis=1)

In [ ]:
all_ab_visits_no_outliers_df.head(5)

# 3. Normalisation

In [ ]:
all_ab_visits_no_outliers_df['visitType'] = all_ab_visits_no_outliers_df['deviceType'] + '-' +\
                                            all_ab_visits_no_outliers_df['channel'] + '-' +\
                                            all_ab_visits_no_outliers_df['bin']

In [ ]:
all_ab_visits_no_outliers_df.head(5)

In [ ]:
all_ab_visits_no_outliers_df.drop(columns = ['outlier'], inplace = True)
all_ab_visits_no_outliers_df.drop(columns = ['z-score'], inplace = True)

In [ ]:
all_ab_visits_no_outliers_df['actualSavings'] = all_ab_visits_no_outliers_df ['maximumSavingsSeen'] *\
                                                all_ab_visits_no_outliers_df ['switched']

In [ ]:
all_ab_visits_no_outliers_df.head(20)

# 4. Create KPI lookup table for visit types

In [ ]:
all_a_visits_df = all_ab_visits_no_outliers_df [all_ab_visits_no_outliers_df ['experimentBucket'] == 'A' ]
all_b_visits_df = all_ab_visits_no_outliers_df [all_ab_visits_no_outliers_df ['experimentBucket'] == 'B' ]

In [ ]:
#all_a_visits_df.info()

In [ ]:
#all_b_visits_df.info()

In [ ]:
lookup_table_a_df = all_a_visits_df.groupby(['visitType']).sum()
lookup_table_a_df['converted'] = lookup_table_a_df['switched'] / lookup_table_a_df['totalVisits']
lookup_table_a_df

In [ ]:
print (lookup_table_a_df ['totalVisits'].sum())
print (lookup_table_a_df ['switched'].sum())
print (lookup_table_a_df ['switched'].sum()/lookup_table_a_df ['totalVisits'].sum())
print (lookup_table_a_df ['actualSavings'].sum())


In [ ]:
lookup_table_b_df = all_b_visits_df.groupby(['visitType']).sum()
lookup_table_b_df['converted'] = lookup_table_b_df['switched'] / lookup_table_b_df['totalVisits']
lookup_table_b_df

In [ ]:
print (lookup_table_b_df ['totalVisits'].sum())
print (lookup_table_b_df ['switched'].sum())
print (lookup_table_b_df ['switched'].sum()/lookup_table_b_df ['totalVisits'].sum())
print (lookup_table_b_df ['actualSavings'].sum())

In [ ]:
lookup_table_df = lookup_table_a_df.merge(lookup_table_b_df, how = 'left', on = 'visitType')

In [ ]:
lookup_table_df.head()

In [ ]:
lookup_table_df.rename(columns={'maximumSavingsSeen_x': 'maximumSavingsSeen_A',\
                                'switched_x' : 'switched_A',\
                                'totalVisits_x' : 'totalVisits_A',\
                                'actualSavings_x' : 'actualSavings_A',\
                                'converted_x' : 'converted_A',\
                                'maximumSavingsSeen_y' : 'maximumSavingsSeen_B',\
                                'switched_y' : 'switched_B',\
                                'totalVisits_y' : 'totalVisits_B',\
                                'actualSavings_y' : 'actualSavings_B',\
                                'converted_y' : 'converted_B'
                                }, inplace = True)

In [ ]:
lookup_table_df.head()

In [ ]:
lookup_table_df.drop(columns = ['maximumSavingsSeen_A',\
                                'switched_A',\
                                'totalVisits_A',\
                                'actualSavings_A',\
                                'maximumSavingsSeen_B',\
                                'switched_B',\
                                'totalVisits_B',\
                                'actualSavings_B'], inplace = True)

In [ ]:
lookup_table_df

# 5. Model conversion rates for website A & website B

In [ ]:
all_ab_visits_no_outliers_df.head()

In [ ]:
summarised_visits_df = all_ab_visits_no_outliers_df.groupby(['visitType']).sum()

summarised_visits_df.drop(columns = ['switched', 'actualSavings'], inplace = True)

summarised_visits_df

In [ ]:
summarised_visits_with_lookup_table_df = summarised_visits_df.merge(lookup_table_df, how = 'left', on ='visitType')
summarised_visits_with_lookup_table_df.head()

In [ ]:
summarised_visits_with_lookup_table_df ['predicted_converted_A'] = \
        summarised_visits_with_lookup_table_df ['totalVisits'] *\
        summarised_visits_with_lookup_table_df ['converted_A']

summarised_visits_with_lookup_table_df ['predicted_converted_B'] = \
        summarised_visits_with_lookup_table_df ['totalVisits'] *\
        summarised_visits_with_lookup_table_df ['converted_B']

summarised_visits_with_lookup_table_df ['predicted_actual_savings_A'] = \
        summarised_visits_with_lookup_table_df ['maximumSavingsSeen'] *\
        summarised_visits_with_lookup_table_df ['converted_A']

summarised_visits_with_lookup_table_df ['predicted_actual_savings_B'] = \
        summarised_visits_with_lookup_table_df ['maximumSavingsSeen'] *\
        summarised_visits_with_lookup_table_df ['converted_B']


In [ ]:
summarised_visits_with_lookup_table_df.head()

In [ ]:
print (summarised_visits_with_lookup_table_df ['totalVisits'].sum())

In [ ]:
total_visits = summarised_visits_with_lookup_table_df ['totalVisits'].sum()
print (summarised_visits_with_lookup_table_df ['predicted_actual_savings_A'].sum())
print (summarised_visits_with_lookup_table_df ['predicted_converted_A'].sum())
print (summarised_visits_with_lookup_table_df ['predicted_converted_A'].sum()/total_visits)
print ()
print (summarised_visits_with_lookup_table_df ['predicted_actual_savings_B'].sum())
print (summarised_visits_with_lookup_table_df ['predicted_converted_B'].sum())
print (summarised_visits_with_lookup_table_df ['predicted_converted_B'].sum()/total_visits)

# 6. Explore possibility of a website C

In [ ]:
device_analysis_df = all_ab_visits_no_outliers_df.groupby(['experimentBucket','deviceType']).sum()
device_analysis_df ['converted'] = device_analysis_df ['switched'] / device_analysis_df ['totalVisits']
device_analysis_df

In [ ]:
channel_analysis_df = all_ab_visits_no_outliers_df.groupby(['experimentBucket','channel']).sum()
channel_analysis_df ['converted'] = channel_analysis_df ['switched'] / channel_analysis_df ['totalVisits']
channel_analysis_df

In [ ]:
bin_analysis_df = all_ab_visits_no_outliers_df.groupby(['experimentBucket','bin']).sum()
bin_analysis_df ['converted'] = bin_analysis_df ['switched'] / bin_analysis_df ['totalVisits']
bin_analysis_df

# 7. Experiment C

In [ ]:
# Experiment C is an experimental website with the UI's of desktop A and mobile/tablet B

In [ ]:
# the following is an example for websites which are not just responsive and which have a good website A for desktop

summarised_visits_with_lookup_table_df['visitType'] = summarised_visits_with_lookup_table_df.index

summarised_visits_with_lookup_table_df['use_A'] = \
                     summarised_visits_with_lookup_table_df['visitType'].apply(lambda x: 1 if 'desktop' in x  else 0)

summarised_visits_with_lookup_table_df['use_B'] = \
                     summarised_visits_with_lookup_table_df['visitType'].apply(lambda x: 1 if 'desktop' not in x  else 0)

In [ ]:
summarised_visits_with_lookup_table_df.head()

In [ ]:
summarised_visits_with_lookup_table_df['converted_C'] = \
                summarised_visits_with_lookup_table_df['converted_A'] * summarised_visits_with_lookup_table_df['use_A'] +\
                summarised_visits_with_lookup_table_df['converted_B'] * summarised_visits_with_lookup_table_df['use_B'] 

In [ ]:
summarised_visits_with_lookup_table_df.head()

In [ ]:
summarised_visits_with_lookup_table_df ['predicted_converted_C'] = \
        summarised_visits_with_lookup_table_df ['totalVisits'] *\
        summarised_visits_with_lookup_table_df ['converted_C']

summarised_visits_with_lookup_table_df ['predicted_actual_savings_C'] = \
        summarised_visits_with_lookup_table_df ['maximumSavingsSeen'] *\
        summarised_visits_with_lookup_table_df ['converted_C']


In [ ]:
total_visits = summarised_visits_with_lookup_table_df ['totalVisits'].sum()
print (summarised_visits_with_lookup_table_df ['totalVisits'].sum())
print (summarised_visits_with_lookup_table_df ['predicted_actual_savings_C'].sum())
print (summarised_visits_with_lookup_table_df ['predicted_converted_C'].sum())
print (summarised_visits_with_lookup_table_df ['predicted_converted_C'].sum()/total_visits)